In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    li

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
  neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
# location of NYC
latitude = 40.7347
longitude = -74.0048

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
#Accessing Foursquare Api as data source for Venues in different cities

CLIENT_ID = 'STW21VBWLIIYZBDHERKEIV1TOT102BIDKIRAYGPVMCG2DL4B' # your Foursquare ID
CLIENT_SECRET = '2OV1QZ5FLHOIP1YLCCUA0IF1QEX1PPEXMED2ULGMD2RMO32V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: STW21VBWLIIYZBDHERKEIV1TOT102BIDKIRAYGPVMCG2DL4B
CLIENT_SECRET:2OV1QZ5FLHOIP1YLCCUA0IF1QEX1PPEXMED2ULGMD2RMO32V


In [21]:
category_argentinian='4bf58dd8d48988d107941735'
LIMIT=10

In [31]:
#crear un mapa con los restos argentinos
df_NYargvenues=pd.read_csv("/resources/data/reportNYvenues.csv")

In [32]:
df_NYargvenues.head()

,Unnamed: 0,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,0,New York,40.7347,-74.0048,Buenos Aires,40.725228,-73.983285,49cd02b2f964a520a9591fe3,Argentinian Restaurant
1,1,New York,40.7347,-74.0048,Porte√±o,40.750443,-74.002407,4ee17c11490152e4bc508413,Argentinian Restaurant
2,2,New York,40.7347,-74.0048,Estancia 460,40.723761,-74.009572,49bd5b75f964a52070541fe3,Argentinian Restaurant
3,3,New York,40.7347,-74.0048,Balvanera,40.720547,-73.985399,53e02d67498e78c091589343,Argentinian Restaurant
4,4,New York,40.7347,-74.0048,El Almac√©n,40.716832,-73.956618,4a01c477f964a520f9701fe3,Argentinian Restaurant


In [33]:
df_NYargvenues.drop(['Unnamed: 0'], axis=1)

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue ID,Venue Category
0,New York,40.7347,-74.0048,Buenos Aires,40.725228,-73.983285,49cd02b2f964a520a9591fe3,Argentinian Restaurant
1,New York,40.7347,-74.0048,Porte√±o,40.750443,-74.002407,4ee17c11490152e4bc508413,Argentinian Restaurant
2,New York,40.7347,-74.0048,Estancia 460,40.723761,-74.009572,49bd5b75f964a52070541fe3,Argentinian Restaurant
3,New York,40.7347,-74.0048,Balvanera,40.720547,-73.985399,53e02d67498e78c091589343,Argentinian Restaurant
4,New York,40.7347,-74.0048,El Almac√©n,40.716832,-73.956618,4a01c477f964a520f9701fe3,Argentinian Restaurant
5,New York,40.7347,-74.0048,Chimichurri Grill,40.759229,-73.992032,3fd66200f964a5208aea1ee3,Argentinian Restaurant
6,New York,40.7347,-74.0048,La Fusta NJ Restaurant,40.763831,-74.048307,4c3a62ea0a71c9b6a63744c9,Argentinian Restaurant
7,New York,40.7347,-74.0048,Empanadas Cafe,40.738168,-74.030641,56e1fb02cd10c99b7c9f4df7,Empanada Restaurant
8,New York,40.7347,-74.0048,La Pulperia,40.760879,-73.990395,5134ec2ae4b01ac3c88b7c59,Latin American Restaurant
9,New York,40.7347,-74.0048,Delmonico's Kitchen,40.752552,-73.990218,505b7dd364a4335f31352e8c,Steakhouse


In [34]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_NYargvenues['Venue Latitude'], df_NYargvenues['Venue Longitude'], df_NYargvenues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan